In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Input, Dense, Activation, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D, AveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from keras import regularizers, optimizers
from keras.models import Sequential, load_model
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.optimizers import SGD
from keras.regularizers import l2
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Model
import datautils
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'


%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv('./data/train_merge.csv')
val_df = pd.read_csv('./data/val_merge.csv')
train_dir = './data/train_set'
val_dir = './data/val_set'

In [11]:
train_sample = 50000
val_sample = 5000
image_size = 128
batch_size = 32
train_generator = datautils.getDeepTrain(train_df, train_dir, sample=train_sample, image_size=image_size, batch_size=batch_size)
val_generator = datautils.getDeepTest(val_df, val_dir, sample=val_sample, image_size=image_size, batch_size=batch_size)

Found 50000 validated image filenames belonging to 251 classes.
Found 5000 validated image filenames belonging to 251 classes.


## ResNet-18

In [3]:
def conv2d_bn(x, nb_filter, kernel_size, strides=(1, 1), padding='same'):
    """
    conv2d -> batch normalization -> relu activation
    """
    x = Conv2D(nb_filter, kernel_size=kernel_size,
                          strides=strides,
                          padding=padding,
                          kernel_regularizer=regularizers.l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

In [4]:
def shortcut(input, residual):
    input_shape = K.int_shape(input)
    residual_shape = K.int_shape(residual)
    stride_height = int(round(input_shape[1] / residual_shape[1]))
    stride_width = int(round(input_shape[2] / residual_shape[2]))
    equal_channels = input_shape[3] == residual_shape[3]
 
    identity = input
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        identity = Conv2D(filters=residual_shape[3],
                           kernel_size=(1, 1),
                           strides=(stride_width, stride_height),
                           padding="valid",
                           kernel_regularizer=regularizers.l2(0.0001))(input)
 
    return add([identity, residual])

In [5]:
def basic_block(nb_filter, strides=(1, 1)):
    def f(input):
 
        conv1 = conv2d_bn(input, nb_filter, kernel_size=(3, 3), strides=strides)
        residual = conv2d_bn(conv1, nb_filter, kernel_size=(3, 3))
 
        return shortcut(input, residual)
 
    return f

In [6]:
def residual_block(nb_filter, repetitions, is_first_layer=False):
    def f(input):
        for i in range(repetitions):
            strides = (1, 1)
            if i == 0 and not is_first_layer:
                strides = (2, 2)
            input = basic_block(nb_filter, strides)(input)
        return input
    return f

In [13]:
from keras.layers import Input
from keras.layers import Conv2D, MaxPool2D, Dense, BatchNormalization, Activation, add, GlobalAvgPool2D
from keras.models import Model
from keras import regularizers
from keras.utils import plot_model
from keras import backend as K

In [14]:
SHAPE = (image_size, image_size, 3)
num_classes = 251
input_ = Input(shape=SHAPE)
conv1 = conv2d_bn(input_, 64, kernel_size=(7, 7), strides=(2, 2))
pool1 = MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same')(conv1)

conv2 = residual_block(64, 2, is_first_layer=True)(pool1)
conv3 = residual_block(128, 2, is_first_layer=True)(conv2)
conv4 = residual_block(256, 2, is_first_layer=True)(conv3)
conv5 = residual_block(512, 2, is_first_layer=True)(conv4)

pool2 = GlobalAvgPool2D()(conv5)
output_ = Dense(num_classes, activation='softmax')(pool2)

model = Model(inputs=input_, outputs=output_)
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 64, 64, 64)   9472        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 64, 64, 64)   256         conv2d_2[0][0]                   
__________________________________________________________________________________________________
activation_2 (Activation)       (None, 64, 64, 64)   0           batch_normalization_2[0][0]      
__________________________________________________________________________________________________
max_pooli

In [17]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size

In [18]:
model.compile(optimizer='adam', loss="categorical_crossentropy",metrics=["accuracy"])

In [19]:
checkpointer = ModelCheckpoint(filepath='best_model_3class.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('history_3class.log')
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=val_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5,
                    verbose=1,
                    callbacks=[csv_logger, checkpointer]
                   )
model.save('model_trained_3class.hdf5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/5
 645/1562 [===========>..................] - ETA: 5:03:30 - loss: 5.6807 - acc: 0.0204

KeyboardInterrupt: 

In [2]:
model = load_model('ResNet_128_50000.hdf5')









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
train_df = pd.read_csv('./data/train_merge.csv')
val_df = pd.read_csv('./data/val_merge.csv')
train_dir = 'D:\Rice\COMP 540/data/train_set'
val_dir = 'D:\Rice\COMP 540/data/val_set'

In [4]:
X_val, y_val, label = datautils.gettestdata(val_df, val_dir, batch_size=32, sample=len(val_df), class_mode='class')

Found 11994 validated image filenames belonging to 251 classes.


KeyboardInterrupt: 